# VRE Example 10 — Regulatory Analytics Suite (SA-CCR, SA‑CVA, FRTB‑SA)

This notebook elaborates on Example 8 and demonstrates a sequence of regulatory analytics with clearly separated, reproducible sections and inline mathematical context.

Conventions below mirror the typical VRE workflow per section:

- Load a master XML configuration into `VRE.vrea.Parameters` (paths under `Examples/*/Input`).
- Construct and run `VRE.vrea.VREApp`.
- Inspect diagnostics and available reports, then view the key result table (falling back to CSV in `Output/...` when needed).

Mathematical summaries are given using LaTeX.


In [ ]:
from utils import format_report, resolve, list_reports, show_report, describe_portfolio, show_saccr_triplet


## 1) SA‑CCR — Standardised Counterparty Credit Risk (BIS)

Mathematical overview (Basel CRE50/52):

- Exposure at default (EAD):
$$  \mathrm{EAD} = \alpha\, (\mathrm{RC} + \mathrm{PFE}), \quad \alpha = 1.4.  $$

- Replacement Cost (current exposure):
$$  \mathrm{RC} = \max(V - C,\, 0),  $$
where $V$ is the current portfolio value (positive means exposure to the counterparty) and $C$ posted collateral.

- Potential Future Exposure (aggregated add‑on):
$$  \mathrm{PFE} = \mathrm{mult} \cdot \sum_{k \in \text{asset classes}} \mathrm{AddOn}_k,  $$
with supervisory factors and effective notionals driving each $\mathrm{AddOn}_k$; within‑class correlations and hedging/offsetting follow the CRE52 rules.

Workflow per example:
1. Load master XML (market, portfolio, supervisory tables, options).
2. Run SA‑CCR analytic; engine writes CSV/JSON outputs (e.g. `saccr_output_*.csv`, tree JSON) and an in‑memory `saccr` table when configured.
3. Inspect totals (EAD, add‑ons, RC) and drill into netting set/bucket breakdown.



### 1a) SA‑CCR — BIS Example 1 (No Aggregation)


> Netting set 1 consists of three interest rates derivatives: two fixed versus floating interest rate swaps and one purchased physically-settled European swaption. The table below summarises the relevant contractual terms of the three derivatives.
>
> Trade # | Nature | Residual maturity | Base currency | Notional (thousands) | Pay Leg (*) | Receive Leg (*) | Market value (thousands)
> ---|---|---|---|---:|---|---|---:
> 1 | Interest rate swap | 10 years | USD | 10,000 | Fixed | Floating | 30
> 2 | Interest rate swap | 4 years | USD | 10,000 | Floating | Fixed | -20
> 3 | European swaption | 1 into 10 years | EUR | 5,000 | Floating | Fixed | 50
>
> (*) For the swaption, the legs are those of the underlying swap.
>
> All notional amounts and market values in the table are given in USD. All the transactions in the netting set belong to the interest rate asset class. For the calculation of the interest rate add-on, the three trades must be assigned to a hedging set (based on the currency) and to a maturity bucket (based on the end date of the transaction). In this example, the netting set is comprised of two hedging sets, since the trades refer to interest rates denominated in two different currencies (USD and EUR). Within hedging set “USD”, trade 1 falls into the third maturity bucket (>5 years) and trade 2 falls into the second maturity bucket (1–5 years). Trade 3 falls into the third maturity bucket (>5 years) of hedging set “EUR”.

For all the SACCR examples that follow, we have assumed that the bank as provided valuations and inputs via a CRIF file. While this is an optional feature (VRE can price almost any instrument), it provides the benefit more easily aligning with the BIS expected outputs (no precisely simulated market is required). 

In [ ]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Examples/CreditRisk/Input/vre_saccr_BIS_1_noagg_EUR.xml'))



In [ ]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Examples/CreditRisk/Input/vre_saccr_BIS_1_noagg_EUR.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports:')
list_reports(app)
describe_portfolio(app)
show_saccr_triplet(tag_hint='BIS_1_noagg')





### 1b) SA‑CCR — BIS Example 1 (Aggregation)

#### Aggregation overview (SA‑CCR Example 1 — Aggregation)

- Replacement Cost (RC) is computed at netting‑set level using netting benefits and collateral under the applicable CSA.
- Potential Future Exposure (PFE) add‑ons are aggregated by hedging set (e.g., currency) and maturity bucket, then combined with prescribed intra‑ and inter‑bucket correlations (CRE52).
- In this example, the netting set contains USD and EUR interest‑rate trades; USD positions span the 1–5y and >5y buckets; the EUR swaption maps to the >5y bucket in the EUR hedging set. Netting within each hedging set reduces effective notional before supervisory factors are applied.


In [ ]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Examples/CreditRisk/Input/vre_saccr_BIS_1_agg_USD.xml'))



In [ ]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Examples/CreditRisk/Input/vre_saccr_BIS_1_agg_USD.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports:')
list_reports(app)
describe_portfolio(app)
show_saccr_triplet(tag_hint='BIS_1_agg')





### 1e) SA‑CCR — BIS Example 4

> Example 4 consists of the three Example 1 IR trades plus three credit derivatives: one long single-name CDS written on Firm A (rated AA),
one short single-name CDS written on Firm B (rated BBB), and one long CDS index (investment grade)
> 
> Since all derivatives refer to different entities (single names/indices), it is not necessary to
aggregate the trades at the entity level. Thus, the entity-level effective notional is equal to the adjusted
notional times the supervisory delta (the maturity factor is 1 for all three derivatives). A supervisory factor
is assigned to each single-name entity based on the rating of the reference entity (0.38% for AA-rated
firms and 0.54% for BBB-rated firms). For CDS indices, the SF is assigned according to whether the index
is investment or speculative grade; in this example, its value is 0.38% since the index is investment grade.


In [ ]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Examples/CreditRisk/Input/vre_saccr_BIS_4.xml'))



In [ ]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Examples/CreditRisk/Input/vre_saccr_BIS_4.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports:')
list_reports(app)
describe_portfolio(app)
show_saccr_triplet(tag_hint='BIS_4')





### 1f) SA‑CCR — BIS Example 5

> Example 5 consists of the three Example 1 IR trades plus three commodity forward contracts. To calculate the add-on, the trades need to be classified into hedging sets (energy, metals, agricultural and other) and, within each hedging set, into commodity types. For purposes of this calculation, the bank can ignore the basis difference between the WTI and Brent forward contracts since they belong to the same commodity type, “Crude Oil” (unless the national supervisor requires the bank to use a more refined definition of commodity types). Therefore, these contracts can be aggregated into a single effective notional, taking into account each trade’s supervisory delta and maturity factor.

> In addition, unlike the examples above, instead of being unmargined (as assumed in those examples), the trades are subject to a margin agreement with the following specifications:

**Margin Agreement (Example 5)**

| Margin frequency | Threshold | Minimum Transfer Amount (thousands) | Independent Amount (thousands) | Net collateral currently held (thousands) |
|---|---:|---:|---:|---:|
| Weekly | 0 | 5 | 150 | 200 |

> The above table depicts a situation in which the bank received from the counterparty a net independent amount of 150 (taking into account the net amount of initial margin posted by the counterparty and any unsegregated initial margin posted by the bank). The total net collateral currently held by the bank is 200, which includes 50 for variation margin received and 150 for the net independent amount.



> From BCBS 279 (select excerpt):
> 
> Supervisory factors, maturity factors and hedging‑set aggregation yield the add‑on; the netting set EAD then follows EAD = 1.4 × (RC + PFE).


In [ ]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Examples/CreditRisk/Input/vre_saccr_BIS_5.xml'))



In [ ]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Examples/CreditRisk/Input/vre_saccr_BIS_5.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports:')
list_reports(app)
describe_portfolio(app)
show_saccr_triplet(tag_hint='BIS_5')





## 2) SA‑CVA and BA‑CVA — Capital and Reduced CVA

Mathematical overview:

- SA‑CVA capital (Basel MAR50, sensitivities‑based): delta/vega margin by risk class with prescribed risk weights and correlations. For a bucket $b$, a stylised form is:
$$  K_b = \sqrt{\sum_{i,j \in b} S_i\,\rho_{ij}\,S_j} \,, \qquad K = \sqrt{\sum_{b,b'} K_b\,\rho^{(b)}_{bb'}\,K_{b'}}.  $$
Here $S_i$ are CVA sensitivities (delta/vega) mapped to CVA risk factors; $\rho$ denotes intra/inter‑bucket correlations.

- Reduced/BA‑CVA (illustrative): CVA as discounted expectation of loss over default time:
$$  \mathrm{CVA} = (1-\mathrm{RR}) \int_0^T \mathrm{EE}^+(t)\, d\mathrm{PD}(t),  $$
where $\mathrm{EE}^+(t)$ is positive expected exposure, and $\mathrm{PD}$ is default probability; capital proxies/adjustments apply per policy.

Workflow per run:
1. Load the XVA master (`vre_bacva_reduced.xml` or `vre_sacva.xml`).
2. Run; inspect `bacva` or `sacva` reports, plus any intermediate sensitivity tables when configured.



### 2a) BA‑CVA — Reduced

In [ ]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Examples/XvaRisk/Input/vre_bacva_reduced.xml'))



In [ ]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Examples/XvaRisk/Input/vre_bacva_reduced.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports (BA-CVA Reduced):')
list_reports(app)
show_report(app, 'bacva')

describe_portfolio(app)



### 2b) SA‑CVA — Classic (Serial FD / Bump and Revalue)

In [ ]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Examples/XvaRisk/Input/vre_sacva.xml'))



In [ ]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Examples/XvaRisk/Input/vre_sacva.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports (SA-CVA):')
list_reports(app)
show_report(app, 'sacva')

describe_portfolio(app)



### 2c) SA‑CVA — Sensitivities via CG/AAD with JIT Kernel and/or SIMD (NEON/mac only in this version AVX/AVX512 wheel in development)

We switch the CVA sensitivity back‑end to algorithmic differentiation (AAD) within the compute‑graph (CG) pipeline. The capital aggregation is unchanged, but the sensitivity source differs.

Notes:
- AAD reduces bump/revaluation noise and accelerates multi‑factor deltas/vegas.
- Report names remain `sacva` for capital results; sensitivity detail may be reported under dedicated tables if configured.

#### Implementation sketch (ScriptedTrades, reverse‑mode AAD)

- Let $V(\theta)$ denote the CVA loss proxy (capital driver) as a function of market parameters $\theta$ (yield pillars, vol nodes, FX spots, …). We target sensitivities $S_i = \partial V/\partial \theta_i$.
- A ScriptedTrade path payoff uses primitives such as LOGPAY, discount/FX conversions and conditionals. Reverse‑mode AAD traverses this computation graph once to accumulate $\nabla V$ efficiently.

Example: a cashflow logged in currency ccy at observation date $d$ with settlement $p$ and base currency discount numeraire $N(d)$:

$$
\mathrm{LOGPAY}(X,d,p,\text{ccy})
= X\, \frac{ P^{\text{ccy}}(d,p)\, \mathrm{FX}_{\text{ccy}\to\text{base}}(d) }{ N(d) }.
$$

Key partials used by AAD (the term‑structure Jacobians map these to pillar shifts):

- With respect to the log discount factor in ccy:
  $$ \frac{\partial\, \mathrm{LOGPAY}}{\partial\, \ln P^{\text{ccy}}(d,p)} = X\, \frac{ P^{\text{ccy}}(d,p)\, \mathrm{FX}(d) }{ N(d) }. $$
- With respect to FX at date $d$:
  $$ \frac{\partial\, \mathrm{LOGPAY}}{\partial\, \mathrm{FX}_{\text{ccy}\to\text{base}}(d)} = X\, \frac{ P^{\text{ccy}}(d,p) }{ N(d) }. $$
- With respect to the numeraire $N(d)$:
  $$ \frac{\partial\, \mathrm{LOGPAY}}{\partial\, N(d)} = - X\, \frac{ P^{\text{ccy}}(d,p)\, \mathrm{FX}(d) }{ N(d)^2 }. $$

Pathwise exposure and hinge: for $\mathrm{EE}^+(t)=\max(\mathrm{NPV}(t),0)$ the reverse pass uses sub‑gradient $\mathbb{1}_{\{\mathrm{NPV}>0\}}$.

AMC/CG fit: continuation values $\mathbb{E}[V_{t+}\mid\phi_t] \approx \beta_t^{\top}\,\varphi(\phi_t)$. In practice, training (sweep 1) solves for $\beta_t$; the reverse pass (sweep 2) treats $\beta_t$ as fixed and propagates through the basis $\varphi$.

CVA‑to‑capital mapping: raw CVA deltas/vegas $J\,\delta\theta$ are bucketed to SA‑CVA factors via a mapping $M$ and then aggregated (see MAR50): $s = M\,J\,\delta\theta$, followed by the quadratic forms shown in Section 2.

#### GPU execution notes (CG/GPU)

- Reverse‑mode per path/trade parallelises naturally. The ScriptedTrade primitives are fused into kernels; adjoints are accumulated with segmented reductions.
- Memory layout uses structure‑of‑arrays for coalesced access to states and adjoints; mixed precision is possible with guardrails on regression quality.
- Typical toggles (via InputParameters) for device offload:
  - `setXvaCgUseExternalComputeDevice(True)`
  - `setXvaCgExternalComputeDevice("CUDA:0")` (or vendor‑specific)
  - `setXvaCgUseDoublePrecisionForExternalCalculation(True/False)`
  - `setXvaCgKernelMaxArgs(n)` as advanced split control

These settings are encapsulated in the provided master XMLs (e.g., `vre_sacva_cg_ad.xml`, `vre_sacva_cg_gpu.xml`).


In [ ]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Examples/XvaRisk/Input/vre_sacva_cg_ad.xml'))



In [ ]:
import VRE as vre
# SA-CVA — CG/AAD sensitivities
params = vre.vrea.Parameters()
params.fromFile(resolve('Examples/XvaRisk/Input/vre_sacva_cg_ad.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports (SA-CVA CG/AAD):')
list_reports(app)
show_report(app, 'sacva')

describe_portfolio(app)



### 2d) SA‑CVA — Sensitivities via CG/GPU

This variant offloads the CVA sensitivities calculation to a GPU‑enabled backend (when available). Capital calculation is as per MAR50; performance may differ substantially.


In [ ]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Examples/XvaRisk/Input/vre_sacva_cg_gpu.xml'))



In [ ]:
import VRE as vre
# SA-CVA — CG/GPU sensitivities
params = vre.vrea.Parameters()
params.fromFile(resolve('Examples/XvaRisk/Input/vre_sacva_cg_gpu.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports (SA-CVA CG/GPU):')
list_reports(app)
show_report(app, 'sacva')

describe_portfolio(app)



## 3) FRTB‑SA — Sensitivity‑Based Method (SBM) and Simplified

Mathematical overview (Basel MAR21/24):

- Risk charge from sensitivities $s_i$ by risk class (e.g., IR/FX/EQ/CM):
$$  K_b = \sqrt{\max\!\left( \sum_{i,j \in b} s_i\,\rho_{ij}\,s_j,\, 0\right)} \,, \qquad K = \sqrt{\sum_{b,b'} K_b\,\rho^{(b)}_{bb'}\,K_{b'}}.  $$
In practice, curvature (gamma/vega adjustments), concentration thresholds, and reduced correlations apply per standard.

Workflow per run:
1. Load the FRTB master (SBM or Simplified) under `Examples/MarketRisk/Input`.
2. Run; inspect sensitivity and capital summary reports beneath `Output/FRTB_SA/...`.



### 3a) FRTB‑SA — SBM (not available in this version)

In [ ]:
# from utils import display_portfolio_description
# display_portfolio_description(resolve('Examples/MarketRisk/Input/vre_frtb_sa_sbm.xml'))



In [ ]:
# import VRE as vre
# params = vre.vrea.Parameters()
# params.fromFile(resolve('Examples/MarketRisk/Input/vre_frtb_sa_sbm.xml'))
# app = vre.vrea.VREApp(params)
# app.run()
# print('Reports (FRTB-SA SBM):')
# list_reports(app)
# # Display likely report names; CSV fallback will locate files if needed
# for cand in ['frtb_sa', 'frtb', 'market_risk', 'sensitivity']:
#     show_report(app, cand)

# describe_portfolio(app)



### 3b) FRTB‑SA — Simplified

In [ ]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Examples/MarketRisk/Input/vre_frtb_sa_simpl.xml'))



In [ ]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Examples/MarketRisk/Input/vre_frtb_sa_simpl.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports (FRTB-SA Simplified):')
list_reports(app)
for cand in ['frtb_sa', 'frtb', 'market_risk', 'sensitivity']:
    show_report(app, cand)

describe_portfolio(app)

